# Beer or Wine: the impact of climate change on Bamberg's vineyards
# Research Question
### Can Franconia's wine region now include Bamberg due to warmer weather?
Despite the negative effects of climate change, some wine varietals thrive in warmer climates. For instance, Bamberg, once a wine-producing region, could potentially regain its status due to rising temperatures. We compare the climate of nearby Würzburg, at the heart of the Franconian winemaking region, to Bamberg's which is currently just outside of the Franconian winemaking region.

## Data Sources
We chose to use station data from the Deutscher Wetterdienst (DWD) because it has many direct measures of weather (ex. air temperature 2 meters above ground) and is accurate, easily accessible, and the DWD works to ensure historical data is integrated and well-maintained.

The data we'll be using are the following (more information can be found [here](https://wetterdienst.readthedocs.io/en/latest/data/coverage/dwd/observation/monthly.html)):
- QN_4 - quality level of the data in the following columns
- MO_N - monthly mean of cloud cover
- MO_TT - monthly mean of daily temperature means in 2m height
- MO_TX - monthly mean of daily temperature maxima in 2m height
- MO_TN - monthly mean of daily temperature minima in 2m height
- MO_FK - monthly mean of daily wind speed
- MX_TX - monthly maximum of daily temperature maxima in 2m height
- MX_FX - monthly maximum of daily wind speed
- MX_TN - monthly minimum of daily temperature minima in 2m height
- MO_SD_S - monthly sum of sunshine duration
- QN_6 - quality level of the data in the following columns
- MO_RR - monthly sum of precipitation height
- MX_RS - monthly maximum of daily precipitation height

### Datasource1: Deutscher Wetterdienst (DWD) Station 282 Bamberg
* Metadata URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/DESCRIPTION_obsgermany_climate_monthly_kl_historical_en.pdf
* Data URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/monatswerte_KL_00282_18810101_20231231_hist.zip
* Data Type: TXT

This data is from the German weather service DWD at node 282 in Bamberg. We'll be using the air temperature monthly aggregate from 1949 - 2024.

### Datasource2: Deutscher Wetterdienst (DWD) Station 5705 Würzburg
* Metadata URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/DESCRIPTION_obsgermany_climate_monthly_kl_historical_en.pdf
* Data URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/monatswerte_KL_05705_18810101_20231231_hist.zip
* Data Type: TXT

This data is from the German weather service DWD at node 5705 in Würzburg. We'll be using the air temperature monthly aggregate from 1949 - 2024.

### Data Source License
The license information can be found in German [here](https://opendata.dwd.de/LIESMICH.txt) and in English [here](https://www.dwd.de/EN/service/legal_notice/legal_notice.html). The weather data is offered under a [Creative Commons 4.0 license](https://creativecommons.org/licenses/by/4.0/). We will fulfill the obligations to name the data source used in a manner which complies with [these guidelines](https://www.dwd.de/EN/service/legal_notice/templates_dwd_as_source.html?nn=450678).

## Data Pipeline
### Overview
We used Python with the urllib.request library for downloading the dataset from the DWD website, the zipfile library to extract the .txt file from the dowloaded ZIP file, and pandas for temporarily storing and transforming the data.

### Transformations
The following transformation steps were applied:
1. Loading the CSV (in .txt form) into pandas converted data types into appropriate values like int64 and float64
2. The "eor" (end of record) column was removed as that's not necessary for our purposes.
3. Missing or invalid data are marked with -999 in the dataset. We replace these with pd.NA (NA values) since we don't want to be able to compare, numerically, valid and invalid data at all.

### Problems
We first attempted to use Jayvee to create the pipeline but ran into issues with formatting. Specifically, it was too difficult to remove (especially varying amounts of) preceding whitespace from a text file which separated values by with a character. This also prevented us from changing the value types from string to integer or decimal. We were also unable to chain multiple transformations, making this issue too time consuming to resolve with Jayvee.

The issue was resolved when we switched to Python using additional libraries.

### Errors and Updates
The DWD has a fixed format for files and data, so we don't expect any deviation from the current format unless DWD standards change.

We do check to see if the file was downloaded, and the data extracted correctly. If not, the process is halted. It's worthy of note that the DWD also has a standardized file naming structure.

The data we are using is updated regularly (at least monthly). No assumptions were made regarding the size of the data we are processing so additional or updated rows should not affect the pipeline negatively. The addition of an entirely new category of data (a new column) would go unused but should not break the pipeline because of how pandas dataframes work.

## Results and Limitations of the Pipeline

### Structure and Quality
The data is tabularly structured into means of means and extrema (min. and max.) as well as the maximum of extrema by month.

We chose to combine both weather stations into a single database for easier storage and loading. It is trivial to split the data again by station ID if need be.

Accuracy, completeness, consistency and timeliness is ensured by a trusted institution, the DWD.

The data is relevant for the question at hand. However, a major factor which affects the growth of vineyards is soil nutrients. That information is not present in our data.

### Output Format
The output is formatted as a .sqlite file for long-term storage and wide compatibility. We intend to analyze the data using pandas dataframes.

### Reflection and Potential Issues
We intend to compare these categories or features between the two datasets and see how much overlap there is.

We're conflicted on renaming the columns; on the one hand they are well-defined with precise meanings from DWD but on the other hand one needs to become familiar with this naming convention to work with them effectively.

We're also concerned that we'll run into errors when trying to compare periods with missing data. However, because a comparison is not possible, we'll likely drop the rows (months) of the specific columns (features) which cannot be compared. We're unable to do this during the data pipeline because that would create a poorly structured database (some columns will be missing some months that other columns are not missing).